In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields)

In [3]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  724.0


In [4]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  207.0


In [5]:
df[df['Lugar de residencia'] == 'Huelva (capital)']

,Lugar de residencia,Medida,Valor
504,Huelva (capital),Población,143663.0
505,Huelva (capital),Confirmados PCR,207.0
506,Huelva (capital),Confirmados PCR 14 días,31.0
507,Huelva (capital),Confirmados PCR 7 días,14.0
508,Huelva (capital),Total Confirmados,257.0
509,Huelva (capital),Curados,187.0
510,Huelva (capital),Fallecidos,15.0


In [6]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [7]:
print('Huelva (capital)\n')
print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Confirmados PCR 7 días x 1,000 habitantes:  0.09745
Un positivo PCR cada 10262 personas en los últimos 7 días 

Confirmados PCR 14 días x 1,000 habitantes:  0.21578
Un positivo PCR cada 4634 personas en los últimos 14 días
